In [20]:
import pandas as pd
import requests
import os
import zipfile
from os import listdir
from os.path import isfile, join
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [2]:
response = requests.get('https://jazz-yt67mbf5eq-ey.a.run.app')
response.status_code

200

In [4]:
url = 'https://jazz-yt67mbf5eq-ey.a.run.app/implement_som'
counter = 1
path = "learning_rate"

while counter <= 20:
    # Save Jazz and CSV File for transfer
    files = {
    "jazz_file": open('Kohonen-Maps/Jazz_Files/{}/data_dog_domain_file_{}.jazz'.format(path, counter), 'rb'), 
    "csv_file": open('Kohonen-Maps/data_dog_prepared.csv','rb')
    }

    # post Request to URL
    response = requests.post(url, files=files)

    # Save the Response ZIP File
    filename = url.split('/')[-1] + '_result.zip'
    with open(os.path.join('Kohonen-Maps', filename), "wb") as output_file:
        output_file.write(response.content)

    # Unzip the File
    with zipfile.ZipFile('Kohonen-Maps/implement_som_result.zip', 'r') as zip_ref:
        name = zip_ref.namelist()
        zip_ref.extract(name[2]) # --> extract the "dimension reduced" csv file

    # Change the File Names in an numerical Order
    onlyfiles = [f for f in listdir('data/processed') if isfile(join('data/processed', f))]
    old_file = os.path.join('data/processed', onlyfiles[0])
    new_file = os.path.join('data/processed', 'v6.{}_data_dog_KM.csv'.format(counter))
    os.rename(old_file, new_file)

    # Remove the Index Column
    df = pd.read_csv('data/processed/v6.{}_data_dog_KM.csv'.format(counter))
    df = df.drop('Unnamed: 0', axis=1)
    df.to_csv('data/processed/v6.{}_data_dog_KM.csv'.format(counter), index=False)

    counter += 1

<h1>Test-Training-Split for V6</h1>

In [7]:
# empty df for the results
df = pd.DataFrame(columns = ['RFC Accuracy', 'KNC Accuracy']) 
counter = 1
path = "processed"

while counter <= 3:
    df6 = pd.read_csv('data/{}/v6.{}_data_dog_KM.csv'.format(path, counter))
    data = df6.values

    # Splitting the data
    X, y = data[:, :-1], data[:, -1]
    # using the train test split function
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
    ### print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    # Random Forest Classifier:
    model = RandomForestClassifier(random_state=1)
    model.fit(X_train, y_train)
    # make predictions
    yhat = model.predict(X_test)
    # evaluate predictions
    acc = accuracy_score(y_test, yhat)

    # K Nearest Neighbor Classification:
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    # make predictions
    y_pred = knn.predict(X_test)
    # evaluate predictions

    df.loc[counter]=[acc, metrics.accuracy_score(y_test, y_pred)]
    counter += 1

df

,RFC Accuracy,KNC Accuracy
1,0.877551,0.806122
2,0.870968,0.830645
3,0.867647,0.838235


In [26]:
df.to_csv('data/Sigma/Score_results.csv', index=False)

In [18]:
path = "Network_size"
# path = "Sigma"
# path = "learning_rate"

df = pd.read_csv('data/{}/Score_results.csv'.format(path))
df.head(55)

,RFC Accuracy,KNC Accuracy,Network_size
0,0.875000,0.750000,5
1,0.785714,0.714286,10
2,0.811321,0.716981,15
3,0.746032,0.650794,20
4,0.800000,0.738462,25
5,0.606061,0.575758,30
6,0.800000,0.642857,35
7,0.816901,0.704225,40
8,0.708333,0.722222,45
9,0.800000,0.742857,50
